In [2]:
import warnings
import numpy as np
import pandas as pd
import sys
import pickle
import time
import scipy.stats
from scipy.special import logsumexp, comb

import matplotlib.pyplot as plt
import seaborn as sns

import torch 
import torch.nn as nn

warnings.filterwarnings("ignore")
sns.set_style("dark")
inf = float("Inf")

In [5]:
DATA = "/home/cyril/Documents/These/data/"
DATASET = "PF01353"

In [6]:
df = pd.read_csv(f"{DATA}/{DATASET}/amino_acid_genotypes_to_brightness.tsv", sep = "\t")

In [ ]:
"MSKGEELFTG VVPILVELDG DVNGHKFSVS GEGEGDATYG KLTLKFICTT GKLPVPWPTL VTTFSYGVQC FSRYPDHMKQ HDFFKSAMPE GYVQERTIFF KDDGNYKTRA EVKFEGDTLV NRIELKGIDF KEDGNILGHK LEYNYNSHNV YIMADKQKNG IKVNFKIRHN IEDGSVQLAD HYQQNTPIGD GPVLLPDNHY LSTQSALSKD PNEKRDHMVL LEFVTAAGIT HGMDELYK"

In [9]:
df.aaMutations

0                                       NaN
1                                    SA108D
2        SA108D:SN144D:SI186V:SM231T:SL234P
3               SA108D:SY141N:SH146L:SE220G
4                                    SA108G
                        ...                
54020                   SY90N:SN103S:SI169F
54021     SY90N:SQ175L:SL205Q:ST228A:SM231V
54022                   SY90R:SI150N:SN196S
54023                   SY90R:SI165T:SL234P
54024     SY90S:SN103D:SK105E:SF128L:SI150V
Name: aaMutations, Length: 54025, dtype: object

In [14]:
def add_muts(x):
    if type(x) is str:
        [all_muts.append((mut[1], mut[-1], int(mut[2:-1]))) for mut in x.split(":")]

In [15]:
all_muts = []
source_seq = 
df.aaMutations.apply(lambda x : add_muts(x))

0        None
1        None
2        None
3        None
4        None
         ... 
54020    None
54021    None
54022    None
54023    None
54024    None
Name: aaMutations, Length: 54025, dtype: object

In [31]:
l = pd.Series([(x[2],x[0]) for x in all_muts]).unique()
l.sort()
seq = ["-"]*236
for i,x in l:
    seq[i]=x

In [90]:
S = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"

In [104]:
"".join(S[5:-11])

'ELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAA'

In [98]:
S = [c for c in S]

In [92]:
nodes = ["".join(S)]
for x in df.aaMutations.values:
    seq = S.copy()
    if type(x) is not str:
        continue
    for mut in x.split(":"):
        aa, pos = mut[-1], int(mut[2:-1])+1
        seq[pos] = aa
    nodes.append("".join(seq))

In [79]:
len(set(nodes))

54025

In [105]:
df["seq"] = nodes
df.seq = df.seq.apply(lambda s : s[5:-11])

In [106]:
df.seq

0        ELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTG...
1        ELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTG...
2        ELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTG...
3        ELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTG...
4        ELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTG...
                               ...                        
54020    ELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTG...
54021    ELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTG...
54022    ELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTG...
54023    ELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTG...
54024    ELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTG...
Name: seq, Length: 54025, dtype: object

In [96]:
df

,aaMutations,uniqueBarcodes,medianBrightness,std,seq
0,NaN,3645,3.719212,0.106992,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...
1,SA108D,1,1.301030,NaN,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...
2,SA108D:SN144D:SI186V:SM231T:SL234P,1,1.301031,NaN,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...
3,SA108D:SY141N:SH146L:SE220G,1,1.301189,NaN,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...
4,SA108G,2,3.708478,0.020613,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...
...,...,...,...,...,...
54020,SY90N:SN103S:SI169F,1,1.552090,NaN,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...
54021,SY90N:SQ175L:SL205Q:ST228A:SM231V,1,1.301030,NaN,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...
54022,SY90R:SI150N:SN196S,1,1.301030,NaN,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...
54023,SY90R:SI165T:SL234P,1,1.301517,NaN,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...
